### ESMA 4016
### k-nn aplicado a datos de reconocimiento de digitos
### Edgar Acuna

In [ ]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn import datasets
from skimage import exposure
import numpy as np
import imutils
import cv2
import matplot.pyplot as plt
%matplotlib inline

Los datos originales estan disponibles en la UCI aqui se usa solo una parte de ellos que vienen con scikit-learn

In [ ]:
# load el MNIST digits dataset
mnist = datasets.load_digits()
print (mnist.data)

In [ ]:
# Graficando  una de las imagenes del digito 0
import matplotlib.pyplot as plt 
plt.gray() 
plt.matshow(mnist.images[0]) 
plt.show() 

In [ ]:
# Training and testing split,
# 75% para training y 25% for testing
(trainData, testData, trainLabels, testLabels) = train_test_split(np.array(mnist.data), mnist.target, test_size=0.25, random_state=42)

# Mostrando los tamanos de cada data split
print("training data points: {}".format(len(trainLabels)))
print("testing data points: {}".format(len(testLabels)))


In [ ]:
# Trataremos de encontrar por inspeccion el mejor k de acuerdo a la precision que da el  clasificador
kVals = range(1, 30, 2)
accuracies = []

# loop over kVals
for k in xrange(1, 30, 2):
    # train the classifier with the current value of `k`
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(trainData, trainLabels)

    # evaluate the model and print the accuracies list
    score = model.score(testData, testLabels)
    print("k=%d, accuracy=%.2f%%" % (k, score * 100))
    accuracies.append(score)


In [ ]:
# Hallando el k para el cual la precision es mayor
i = np.argmax(accuracies)
print("k=%d dio la mayor precision de %.2f%%" % (kVals[i],
    accuracies[i] * 100))


In [ ]:
# Aplicamos otra vez k-nn pero con el mejor clasificador
model = KNeighborsClassifier(n_neighbors=kVals[i])
model.fit(trainData, trainLabels)

# Predict labels for the test set
predictions = model.predict(testData)

# Evaluando el rendimiento del clasificador para cada uno de lpos digitos
print("EVALUATION ON TESTING DATA")
print(classification_report(testLabels, predictions))

# Algunos digitso son clasificados con 100% de precision


In [ ]:
# Cotejando las predicciones contra las imagenes 
# Eligiendo al azar una imagen
i=np.random.randint(0, high=len(testLabels))
image=testData[i].reshape(1,-1)
prediction = model.predict(image)[0]

# Convirtiendo la imagen de un arreglo de dimension 64 a una imagen de dimension 8x8 compatibel con OpenVC
# rescalando luego a una imagen de 32x32 pixels para que se vea mejor
image = image.reshape((8, 8)).astype("uint8")
image = exposure.rescale_intensity(image, out_range=(0, 255))
image = imutils.resize(image, width=32, inter=cv2.INTER_CUBIC)

# Mostrando la prediccion
print("El digito de la imagen es: {}".format(prediction))
plt.gray() 
plt.matshow(image) 
plt.show() 